Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# Set Cover problem

See: https://en.wikipedia.org/wiki/Set_cover_problem

In [2]:
import numpy as np
from numpy.typing import NDArray
from icecream import ic

from tqdm.auto import tqdm
from itertools import accumulate
from matplotlib import pyplot as plt

## Helper Functions

In [3]:
def valid(solution) -> np.bool:
    """Checks wether solution is valid (ie. covers all universe)"""
    return np.all(np.logical_or.reduce(SETS[solution]))


def cost(solution)-> np.float64:
    """Returns the cost of a solution (to be minimized)"""
    return COSTS[solution].sum()

In [23]:
def fitness(
    solution: NDArray[np.bool],
    penalty: dict[str, float] = {
        "redundancy": 5.26,
    },
) -> np.float64:
    # Calculate how often each element in the universe is covered
    element_coverage: NDArray[np.int16] = SETS[solution].sum(axis=0)

    # Penalize for redundant coverage: penalize elements covered more than once
    redundancy_penalty = np.sum((element_coverage[element_coverage > 1]) ** penalty["redundancy"])

    # Calculate the fitness value
    if valid(solution):
        return -cost(solution) - redundancy_penalty
    else:
        return np.float64(-np.inf)


def mutate(
    solution: NDArray[np.bool],
    penalty: dict[str, float] = {
        "redundancy": 5.26,
    },
) -> NDArray[np.bool]:
    new_sol: NDArray[np.bool] = solution.copy()
    while not valid(new_sol):
        i: int = rng.integers(0, NUM_SETS)
        new_sol[i] = not new_sol[i]
    return new_sol


def fit(
    solution: NDArray[np.bool],
    steps: int = 200,
    penalty: dict[str, float] = {
        "redundancy": 5.26,
    },
    plot: bool = False,
) -> NDArray[np.bool]:
    # Both history and progress bar are copied from here:
    # https://github.com/squillero/computational-intelligence/blob/master/2024-25/multiple-knapsack.ipynb

    # history appending was a slow operation, if you want to plot the history, you can uncomment it along with the plot code
    # history: list[np.float64] = [fitness(solution)]
    # initial mutation to reduce the number of copy operations

    # history.append(fitness(new_solution))
    new_solution: NDArray[np.bool] = np.zeros(NUM_SETS, dtype=bool)

    for _ in tqdm(range(steps)):
        new_solution: NDArray[np.bool] = mutate(solution, penalty)
        if fitness(new_solution) > fitness(solution):
            solution = new_solution
        # history.append(fitness(new_solution))

    # if plot:
    #     plt.figure(figsize=(14, 8))
    #     plt.plot(
    #         range(len(history)),
    #         list(accumulate(history, max)),  # type: ignore
    #         color="red",
    #     )
    #     plt.scatter(range(len(history)), history, marker=".") # type: ignore
    return new_solution

## Solving for different Configuration

In [24]:
for instance in tqdm(range(1, 7)):
    # DON'T EDIT THESE LINES!
    match instance:
        case 1:
            UNIVERSE_SIZE = 100
            NUM_SETS = 10
            DENSITY = 0.2
        case 2:
            UNIVERSE_SIZE = 1_000
            NUM_SETS = 100
            DENSITY = 0.2

        case 3:
            UNIVERSE_SIZE = 10_000
            NUM_SETS = 1_000
            DENSITY = 0.2
        case 4:
            UNIVERSE_SIZE = 100_000
            NUM_SETS = 10_000
            DENSITY = 0.1
        case 5:
            UNIVERSE_SIZE = 100_000
            NUM_SETS = 10_000
            DENSITY = 0.2
        case 6:
            UNIVERSE_SIZE = 100_000
            NUM_SETS = 10_000
            DENSITY = 0.3
        case _:
            raise ValueError("How did you get here?")
        
    rng = np.random.Generator(np.random.PCG64([UNIVERSE_SIZE, NUM_SETS, int(NUM_SETS * DENSITY)]))

    SETS = np.random.random((NUM_SETS, UNIVERSE_SIZE)) < DENSITY
    for s in range(UNIVERSE_SIZE):
        if not np.any(SETS[:, s]):
            SETS[np.random.randint(NUM_SETS), s] = True
    COSTS = np.pow(SETS.sum(axis=1), 1.1)

    # Starting with a random solution with random 50% of the sets
    random_solution: NDArray[np.bool] = rng.random(NUM_SETS) < 0.5
    solution: NDArray[np.bool] = fit(random_solution, steps=100)

    ic(
        instance,
        UNIVERSE_SIZE,
        NUM_SETS,
        DENSITY,
        fitness(random_solution),
        cost(random_solution),
        valid(random_solution),
        fitness(solution),
        cost(solution),
        valid(solution),
    )

    print(f"|{instance}|{UNIVERSE_SIZE}|{NUM_SETS}|{DENSITY}|{fitness(solution):.2f}|{cost(solution):.2f}|{valid(solution)}|")

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

ic| instance: 1
    UNIVERSE_SIZE: 100
    NUM_SETS: 10
    DENSITY: 0.2
    fitness(random_solution): np.float64(-inf)
    cost(random_solution): np.float64(168.2342639444425)
    valid(random_solution): np.False_
    fitness(solution): np.float64(-28916.550496632357)
    cost(solution): np.float64(278.0248009203271)
    valid(solution): np.True_


|1|100|10|0.2|-28916.55|278.02|True|


  0%|          | 0/100 [00:00<?, ?it/s]

...

ic| instance: 5
    UNIVERSE_SIZE: 100000
    NUM_SETS: 10000
    DENSITY: 0.2
    fitness(random_solution): np.float64(-6.000661447640608e+20)
    cost(random_solution): np.float64(268547654.4317805)
    valid(random_solution): np.True_
    fitness(solution): np.float64(-6.000661447640608e+20)
    cost(solution): np.float64(268547654.4317805)
    valid(solution): np.True_


|5|100000|10000|0.2|-600066144764060762112.00|268547654.43|True|


  0%|          | 0/100 [00:00<?, ?it/s]

ic| instance: 6
    UNIVERSE_SIZE: 100000
    NUM_SETS: 10000
    DENSITY: 0.3
    fitness(random_solution): np.float64(-5.303792096180068e+21)
    cost(random_solution): np.float64(423510822.07619506)
    valid(random_solution): np.True_
    fitness(solution): np.float64(-5.303792096180068e+21)
    cost(solution): np.float64(423510822.07619506)
    valid(solution): np.True_


|6|100000|10000|0.3|-5303792096180067893248.00|423510822.08|True|
